In [18]:
# import de bibliothèque du systemes
import os
import glob
import shutil
import random
import pathlib

# import librairies de base
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

# import de librairie de tensorflow

import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix


import tensorflow
print(tensorflow.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# import outils de tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory 

2.4.1


In [19]:
base_dir = os.getcwd()
os.chdir(base_dir)
print("current directory " , base_dir)

current directory  C:\Users\Mehdi\Desktop\Deep learning Individuel


### Telecharger le datasets

In [20]:
data_directory = base_dir +'/Images'
dataset_url = "http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar"
data_directory = tf.keras.utils.get_file('Images', origin=dataset_url, untar = True)
data_directory = pathlib.Path(data_directory)

### Creation de datasets

In [21]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
TEST_RATIO = 0.1
IMAGES_DIRECTORY = data_directory

In [22]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(IMAGES_DIRECTORY,
                                                    image_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
                                                    label_mode="categorical",
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    validation_split =  TEST_RATIO,
                                                    subset="training",                                                  
                                                    seed=222,
                                                    shuffle=True)

Found 20580 files belonging to 120 classes.
Using 18522 files for training.


In [23]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(IMAGES_DIRECTORY,
                                                    image_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
                                                    label_mode="categorical",
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    validation_split =  TEST_RATIO,
                                                    subset="validation",
                                                    seed=222,
                                                    shuffle=True)

Found 20580 files belonging to 120 classes.
Using 2058 files for validation.


In [ ]:
### Detection de chien

In [24]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tqdm import tqdm

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

# preprocessing images
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

#preprocessing images
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)
 
    
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151))


In [54]:
dog_detector('test.jpg')

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node resnet50/conv1_conv/Conv2D (defined at <ipython-input-24-d958416194cc>:27) ]] [Op:__inference_predict_function_13908]

Function call stack:
predict_function


In [ ]:
#Create a CNN to Classify Dog Breeds (from Scratch)

In [31]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential()

###architecture of the model.
model.add(Conv2D(16, kernel_size=2, activation='relu', padding='valid', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(Conv2D(32, kernel_size=2, activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(Conv2D(64, kernel_size=2, activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

# Adding two additional stages
# Stage 1: Conv2D with 128 filters and Pooling layer
model.add(Conv2D(128, kernel_size=2, activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

# Stage 2: Conv2D with 256 filters and Pooling layer
model.add(Conv2D(256, kernel_size=2, activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(GlobalAveragePooling2D())

# Dropout layer for regularization
model.add(Dropout(0.4))

model.add(Dense(units=120, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 54, 54, 64)        8256      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 128)      

In [32]:
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [33]:
epochs=30

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=0.1,
    verbose=1
)


Epoch 1/30


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_2/conv2d_10/Relu (defined at <ipython-input-33-bb33719065fb>:3) ]] [Op:__inference_train_function_15111]

Function call stack:
train_function


In [35]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import numpy as np
# new model
AlexNet_model = Sequential()

# Conv2D C1
AlexNet_model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'))
# Max Pooling S2
AlexNet_model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Conv2D C3
AlexNet_model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
# Max Pooling S4
AlexNet_model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Conv2D C5
AlexNet_model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

# Conv2D C6
AlexNet_model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

# Conv2D C7
AlexNet_model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#AlexNet_model.add(BatchNormalization())

# Max Pooling S8
AlexNet_model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Flatten for FC
AlexNet_model.add(Flatten())

# FC F9
AlexNet_model.add(Dense(4096, input_shape=(224*224*3,), activation='relu'))
# Dropout
AlexNet_model.add(Dropout(0.5))

# FC F9
AlexNet_model.add(Dense(4096,activation='relu'))
# Dropout
AlexNet_model.add(Dropout(0.5))

# Output
AlexNet_model.add(Dense(120, activation='softmax'))

AlexNet_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 26, 26, 256)       614656    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 12, 12, 384)       1327488   
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 12, 12, 256)      

In [46]:
AlexNet_model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [47]:
history = AlexNet_model.fit(
    train_dataset,
    epochs=30,
    validation_data=0.1,
    verbose=1
)


Epoch 1/30


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_4/conv2d_20/Relu (defined at <ipython-input-47-b33dfbf220a4>:1) ]] [Op:__inference_train_function_16985]

Function call stack:
train_function


In [48]:
bottleneck_features = np.load('bottleneck_features/DogVGG16Data.npz')

FileNotFoundError: [Errno 2] No such file or directory: 'bottleneck_features/DogVGG16Data.npz'

In [51]:
from sklearn import datasets  
from sklearn.datasets import load_files       
from tensorflow.keras.utils import to_categorical
import numpy as np
from glob import glob



In [52]:
# Define functions to load train, test and validation data sets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets



In [53]:
# Load train, test and validation data sets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')



FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'dogImages/train'